# Minimal LangGraph Agent over Accelerator RAG

This notebook shows a minimal LangGraph-based agent that:

- Accepts a user question.
- Calls the accelerator `/ask` endpoint in a **retrieval node**.
- Uses an LLM in a **generation node** to produce the final answer.

It is intentionally simple and mirrors the Evaluation Studio example at a smaller scale.


In [ ]:
# Install dependencies (run once per environment)
!pip install -q langgraph langchain-core langchain-openai requests


In [ ]:
import os
from typing_extensions import TypedDict
from typing import Dict, Any

import requests
from langgraph.graph import START, END, StateGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

ACCELERATOR_API_URL = os.getenv("ACCELERATOR_API_URL", "http://localhost:8000/ask")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or input("OPENAI_API_KEY: ")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=OPENAI_API_KEY)


In [ ]:
class GraphState(TypedDict):
    """State for our simple LangGraph app.

    - input_text: user question.
    - rag_answer: raw response from accelerator RAG service.
    - final_answer: polished answer from LLM.
    """

    input_text: str
    rag_answer: Dict[str, Any]
    final_answer: str


## Define Nodes

1. `rag_node` – calls the accelerator `/ask` endpoint.
2. `generation_node` – uses the LLM to polish the answer.


In [ ]:
def rag_node(state: GraphState) -> Dict[str, Any]:
    question = state["input_text"]
    resp = requests.post(ACCELERATOR_API_URL, json={"question": question}, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    return {"rag_answer": data}


def generation_node(state: GraphState) -> Dict[str, Any]:
    question = state["input_text"]
    rag_answer = state.get("rag_answer", {})
    answer = rag_answer.get("answer") or rag_answer.get("result") or "(no answer field)"
    citations = rag_answer.get("citations") or rag_answer.get("chunks") or []

    prompt = ChatPromptTemplate.from_template(
        "You are a helpful assistant.\n"
        "Here is a draft answer from a RAG service and its citations.\n\n"
        "Question: {question}\n\n"
        "Draft answer: {draft}\n\n"
        "Citations: {citations}\n\n"
        "Please rewrite the answer in a clear, concise way. If the draft looks incomplete, say so."
    )

    formatted = prompt.invoke({
        "question": question,
        "draft": answer,
        "citations": str(citations),
    })
    result = llm.invoke(formatted)
    return {"final_answer": result.content}


## Build and Compile the Graph


In [ ]:
graph = StateGraph(GraphState)
graph.add_node("rag_node", rag_node)
graph.add_node("generation_node", generation_node)

graph.add_edge(START, "rag_node")
graph.add_edge("rag_node", "generation_node")
graph.add_edge("generation_node", END)

app = graph.compile()


## Run a Test Question


In [ ]:
state = {"input_text": "Explain what RAG is and why it's useful."}
result = app.invoke(state)
result
